# ソフトマックス関数と全結合層

このノートブックでは、CNNの最終段階である**全結合層**と**ソフトマックス関数**を学びます。

## 目次
1. 数学的基礎（確率、ネイピア数、総和記号）
2. 全結合層（Fully Connected Layer）
3. ソフトマックス関数
4. パラメータ数の計算
5. 完全なCNN分類の流れ

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 1. 数学的基礎（図3.15）

ソフトマックス関数を理解するために、まず3つの数学概念を確認します。

### 1.1 確率

- 0から1の間の値
- すべての確率の合計は1
- 例: サイコロの目が1になる確率 = 1/6 ≈ 0.167

### 1.2 ネイピア数 $e$

$$e \approx 2.71828...$$

- 自然対数の底
- 円周率πのように無限に続く無理数
- 微分しても変わらない特別な性質: $(e^x)' = e^x$

### 1.3 総和記号 $\Sigma$

$$\sum_{k=1}^{n} a_k = a_1 + a_2 + \cdots + a_n$$

「$k=1$から$n$まで、$a_k$をすべて足し合わせる」という意味

In [ ]:
# ネイピア数の確認
print("=== ネイピア数 e ===")
print(f"np.e = {np.e}")
print(f"e^0 = {np.exp(0)}")
print(f"e^1 = {np.exp(1):.5f}")
print(f"e^2 = {np.exp(2):.5f}")

print("\n=== 総和記号 Σ の例 ===")
a = [1, 2, 3, 4, 5]
print(f"a = {a}")
print(f"Σa = {sum(a)}  (1+2+3+4+5)")

In [ ]:
# e^x のグラフ
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 左: e^x のグラフ
ax = axes[0]
x = np.linspace(-2, 3, 100)
y = np.exp(x)
ax.plot(x, y, 'b-', linewidth=2, label='$y = e^x$')
ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)
ax.axhline(y=1, color='gray', linestyle='--', alpha=0.5)
ax.scatter([0, 1, 2], [np.exp(0), np.exp(1), np.exp(2)], color='red', s=100, zorder=5)
ax.annotate(f'e^0=1', xy=(0, 1), xytext=(0.5, 1.5), fontsize=10)
ax.annotate(f'e^1≈2.72', xy=(1, np.e), xytext=(1.3, np.e+1), fontsize=10)
ax.annotate(f'e^2≈7.39', xy=(2, np.exp(2)), xytext=(2.2, np.exp(2)+1), fontsize=10)
ax.set_xlabel('x')
ax.set_ylabel('$e^x$')
ax.set_title('ネイピア数の指数関数 $e^x$', fontsize=11)
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(-1, 15)

# 右: e^x の特徴
ax = axes[1]
ax.text(0.5, 0.9, '$e^x$ の重要な性質', fontsize=14, fontweight='bold', 
        ha='center', transform=ax.transAxes)
ax.text(0.1, 0.7, '1. 常に正の値: $e^x > 0$（どんなxでも）', fontsize=11, transform=ax.transAxes)
ax.text(0.1, 0.55, '2. x=0のとき1: $e^0 = 1$', fontsize=11, transform=ax.transAxes)
ax.text(0.1, 0.4, '3. xが大きいほど急激に増加', fontsize=11, transform=ax.transAxes)
ax.text(0.1, 0.25, '4. 微分しても変わらない: $(e^x)\' = e^x$', fontsize=11, transform=ax.transAxes)
ax.text(0.1, 0.1, '→ ソフトマックス関数で大活躍！', fontsize=11, color='blue', transform=ax.transAxes)
ax.axis('off')

plt.tight_layout()
plt.show()

## 2. 全結合層（Fully Connected Layer）（図3.16）

畳み込み層・プーリング層で抽出した特徴を、最終的な分類結果に変換するのが**全結合層**です。

### CNNの処理フロー

```
入力画像 → 畳み込み → プーリング → 畳み込み → プーリング → 平坦化 → 全結合層 → 出力
(24×24)    (20×20×2)   (10×10×2)   (6×6×4)    (3×3×4)     (36)       (10)      (確率)
```

### 平坦化（Flatten）

プーリング後の3次元データ（例: 4×4×4）を1次元に並べ替えます：

$$4 \times 4 \times 4 = 64 \text{個のニューロン}$$

### 全結合層の役割

- 入力: 64個の特徴（平坦化後）
- 出力: 10個のクラス（0〜9の数字）
- すべての入力が、すべての出力に接続される

In [ ]:
# 図3.16: 全結合層の可視化

fig, ax = plt.subplots(figsize=(14, 8))

# 入力層（64ニューロン - 省略して表示）
input_neurons = 8  # 表示用（実際は64）
output_neurons = 10

# 位置設定
input_x = 2
output_x = 10
y_start = 1
y_spacing_in = 0.8
y_spacing_out = 0.7

# 入力ニューロンを描画
input_positions = []
for i in range(input_neurons):
    if i == 4:  # 省略記号を入れる
        y = y_start + i * y_spacing_in
        ax.text(input_x, y, '⋮', fontsize=20, ha='center', va='center')
        continue
    y = y_start + i * y_spacing_in
    circle = plt.Circle((input_x, y), 0.25, color='steelblue', ec='black', linewidth=1)
    ax.add_patch(circle)
    input_positions.append((input_x, y))
    label = f'$x_{{{i+1}}}$' if i < 4 else f'$x_{{{60+i-4}}}$'
    ax.text(input_x - 0.6, y, label, fontsize=10, ha='right', va='center')

# 出力ニューロンを描画
output_positions = []
for i in range(output_neurons):
    y = y_start + i * y_spacing_out + 0.5
    circle = plt.Circle((output_x, y), 0.25, color='coral', ec='black', linewidth=1)
    ax.add_patch(circle)
    output_positions.append((output_x, y))
    ax.text(output_x + 0.6, y, f'{i}', fontsize=12, ha='left', va='center', fontweight='bold')

# 結合線を描画（一部のみ）
for i, (ix, iy) in enumerate(input_positions):
    for j, (ox, oy) in enumerate(output_positions):
        alpha = 0.1 if (i + j) % 3 != 0 else 0.3
        ax.plot([ix + 0.25, ox - 0.25], [iy, oy], 'gray', alpha=alpha, linewidth=0.5)

# ラベル
ax.text(input_x, y_start - 1, '入力層\n(64ニューロン)', fontsize=12, ha='center', fontweight='bold')
ax.text(output_x, y_start - 1, '出力層\n(10ニューロン)', fontsize=12, ha='center', fontweight='bold')

# 重みとバイアスの説明
ax.annotate('', xy=(6, 4), xytext=(4, 5),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))
ax.text(5, 5.5, '重み $w_{i,j}$\n(各接続に1つ)', fontsize=10, ha='center', color='green')

ax.text(output_x + 2, 4, 'バイアス $b_k$\n(各出力に1つ)', fontsize=10, ha='left', color='purple')

# タイトル
ax.set_title('図3.16: 全結合層（Fully Connected Layer）\n64個の特徴 → 10個のクラス', 
             fontsize=14, fontweight='bold')

ax.set_xlim(0, 14)
ax.set_ylim(-1, 9)
ax.set_aspect('equal')
ax.axis('off')

plt.tight_layout()
plt.show()

## 3. ソフトマックス関数（図3.15）

全結合層の出力を**確率**に変換する関数です。

### 定義

$$\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_{k=1}^{n} e^{x_k}}$$

### わかりやすく言うと

1. 各入力に $e^x$ を適用（すべて正の値に）
2. それぞれを全体の合計で割る（合計が1になる）

### 例: 3つの入力 [2, 1, 0]

$$\text{softmax}([2, 1, 0]) = \frac{[e^2, e^1, e^0]}{e^2 + e^1 + e^0} = \frac{[7.39, 2.72, 1]}{11.11} = [0.67, 0.24, 0.09]$$

合計: $0.67 + 0.24 + 0.09 = 1.00$ ✓

In [ ]:
def softmax(x):
    """ソフトマックス関数
    
    Args:
        x: 入力配列
    Returns:
        確率分布（合計が1になる配列）
    """
    # オーバーフロー対策: 最大値を引く
    x_shifted = x - np.max(x)
    exp_x = np.exp(x_shifted)
    return exp_x / np.sum(exp_x)

# 例: 3つの入力
x = np.array([2, 1, 0])
print("=== ソフトマックス関数の計算例 ===")
print(f"入力: x = {x}")
print(f"")
print("ステップ1: e^x を計算")
exp_x = np.exp(x)
print(f"  e^2 = {exp_x[0]:.4f}")
print(f"  e^1 = {exp_x[1]:.4f}")
print(f"  e^0 = {exp_x[2]:.4f}")
print(f"")
print(f"ステップ2: 合計を計算")
print(f"  Σe^x = {exp_x[0]:.4f} + {exp_x[1]:.4f} + {exp_x[2]:.4f} = {np.sum(exp_x):.4f}")
print(f"")
print(f"ステップ3: 各値を合計で割る")
probs = softmax(x)
for i, (e, p) in enumerate(zip(exp_x, probs)):
    print(f"  softmax(x_{i}) = {e:.4f} / {np.sum(exp_x):.4f} = {p:.4f} ({p*100:.1f}%)")
print(f"")
print(f"確率の合計: {np.sum(probs):.4f} ✓")

In [ ]:
# ソフトマックスの可視化

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# 例1: [2, 1, 0]
x1 = np.array([2, 1, 0])
p1 = softmax(x1)

ax = axes[0]
bars = ax.bar(['x₀=2', 'x₁=1', 'x₂=0'], p1, color=['coral', 'steelblue', 'gray'])
ax.set_ylim(0, 1)
ax.set_ylabel('確率')
ax.set_title(f'入力: [2, 1, 0]\n最大値が最も高い確率', fontsize=11)
for bar, prob in zip(bars, p1):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{prob:.2f}', ha='center', fontsize=11)

# 例2: [5, 5, 5] - すべて同じ
x2 = np.array([5, 5, 5])
p2 = softmax(x2)

ax = axes[1]
bars = ax.bar(['x₀=5', 'x₁=5', 'x₂=5'], p2, color=['coral', 'steelblue', 'gray'])
ax.set_ylim(0, 1)
ax.set_ylabel('確率')
ax.set_title(f'入力: [5, 5, 5]\nすべて同じ → 均等な確率', fontsize=11)
for bar, prob in zip(bars, p2):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{prob:.2f}', ha='center', fontsize=11)

# 例3: [10, 0, 0] - 差が大きい
x3 = np.array([10, 0, 0])
p3 = softmax(x3)

ax = axes[2]
bars = ax.bar(['x₀=10', 'x₁=0', 'x₂=0'], p3, color=['coral', 'steelblue', 'gray'])
ax.set_ylim(0, 1)
ax.set_ylabel('確率')
ax.set_title(f'入力: [10, 0, 0]\n差が大きい → ほぼ100%', fontsize=11)
for bar, prob in zip(bars, p3):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{prob:.2f}', ha='center', fontsize=11)

plt.suptitle('ソフトマックス関数: 入力値を確率に変換', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

## 4. パラメータ数の計算（図3.17）

全結合層のパラメータ数を計算してみましょう。

### 設定
- 入力: 64ニューロン（4×4×4の特徴マップを平坦化）
- 出力: 10ニューロン（0〜9の数字分類）

### 計算

$$\text{重みパラメータ} = 64 \times 10 = 640$$
$$\text{バイアス} = 10$$
$$\text{合計} = 640 + 10 = 650$$

In [ ]:
# パラメータ数の計算

input_neurons = 64   # 4×4×4 を平坦化
output_neurons = 10  # 0〜9の数字

weights = input_neurons * output_neurons
biases = output_neurons
total = weights + biases

print("=== 全結合層のパラメータ数（図3.17）===")
print(f"")
print(f"入力ニューロン数: {input_neurons}  (4×4×4 を平坦化)")
print(f"出力ニューロン数: {output_neurons} (0〜9の数字)")
print(f"")
print(f"重みパラメータ: {input_neurons} × {output_neurons} = {weights}")
print(f"バイアス: {output_neurons}")
print(f"────────────────────")
print(f"合計: {total} パラメータ")

In [ ]:
# 図3.17: パラメータ数の可視化

fig, ax = plt.subplots(figsize=(12, 6))

# 左側: 重み行列の表現
weight_matrix = np.random.randn(10, 64) * 0.5
im = ax.imshow(weight_matrix, aspect='auto', cmap='RdBu', vmin=-1, vmax=1)

ax.set_xlabel('入力 (64ニューロン)', fontsize=11)
ax.set_ylabel('出力 (10クラス: 0〜9)', fontsize=11)
ax.set_yticks(range(10))
ax.set_yticklabels([f'クラス {i}' for i in range(10)])

# カラーバー
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('重みの値')

# パラメータ数の注釈
ax.set_title('全結合層の重み行列 (64×10 = 640パラメータ) + バイアス10 = 650パラメータ', 
             fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("各行（クラス）には64個の重みがある")
print("→ 64個の入力特徴それぞれとの接続強度")

## 5. 完全なCNN分類の流れ

入力画像から最終的な分類結果（確率）までの流れをまとめます。

In [ ]:
# 完全なCNN分類の可視化

fig, ax = plt.subplots(figsize=(16, 8))

# 各層の情報
layers = [
    {'name': '入力\n24×24×1', 'x': 0, 'color': 'lightgray', 'width': 0.8, 'height': 2.4},
    {'name': '畳み込み1\n+ReLU\n20×20×2', 'x': 2, 'color': 'steelblue', 'width': 0.8, 'height': 2.0},
    {'name': 'プーリング1\n10×10×2', 'x': 4, 'color': 'coral', 'width': 0.6, 'height': 1.0},
    {'name': '畳み込み2\n+ReLU\n6×6×4', 'x': 6, 'color': 'steelblue', 'width': 0.8, 'height': 0.6},
    {'name': 'プーリング2\n4×4×4', 'x': 8, 'color': 'coral', 'width': 0.6, 'height': 0.4},
    {'name': '平坦化\n64', 'x': 10, 'color': 'gold', 'width': 0.3, 'height': 2.5},
    {'name': '全結合\n10', 'x': 12, 'color': 'purple', 'width': 0.3, 'height': 1.5},
    {'name': 'Softmax\n確率', 'x': 14, 'color': 'green', 'width': 0.3, 'height': 1.5},
]

center_y = 3

for i, layer in enumerate(layers):
    from matplotlib.patches import FancyBboxPatch
    
    x = layer['x']
    w = layer['width']
    h = layer['height']
    
    rect = FancyBboxPatch((x - w/2, center_y - h/2), w, h,
                           boxstyle="round,pad=0.05",
                           facecolor=layer['color'], edgecolor='black',
                           linewidth=2, alpha=0.8)
    ax.add_patch(rect)
    
    ax.text(x, center_y - h/2 - 0.4, layer['name'], 
            ha='center', va='top', fontsize=9, fontweight='bold')
    
    # 矢印
    if i < len(layers) - 1:
        next_x = layers[i+1]['x']
        ax.annotate('', xy=(next_x - layers[i+1]['width']/2 - 0.1, center_y),
                    xytext=(x + w/2 + 0.1, center_y),
                    arrowprops=dict(arrowstyle='->', color='black', lw=1.5))

# 最終出力（確率）
probs = softmax(np.array([2.1, 0.5, 8.3, 1.2, 0.1, 0.3, 0.8, 9.5, 0.2, 0.4]))
best_class = np.argmax(probs)

ax.text(15.5, center_y + 1.5, '出力確率', fontsize=11, fontweight='bold')
for i, p in enumerate(probs):
    color = 'red' if i == best_class else 'black'
    weight = 'bold' if i == best_class else 'normal'
    ax.text(15.5, center_y + 1 - i*0.3, f'P({i}): {p:.1%}', 
            fontsize=9, color=color, fontweight=weight)

ax.text(15.5, center_y - 2.5, f'予測: {best_class}', fontsize=14, fontweight='bold', color='red')

ax.set_xlim(-1, 18)
ax.set_ylim(-1, 6)
ax.set_title('CNNによる画像分類の完全な流れ', fontsize=14, fontweight='bold')
ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# PyTorchでの完全な実装例
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    """教科書に基づくシンプルなCNN"""
    
    def __init__(self):
        super().__init__()
        # 畳み込み層1: 1ch → 2ch, 5×5フィルタ
        self.conv1 = nn.Conv2d(1, 2, kernel_size=5)
        # 畳み込み層2: 2ch → 4ch, 5×5フィルタ
        self.conv2 = nn.Conv2d(2, 4, kernel_size=5)
        # 全結合層: 4×4×4=64 → 10
        self.fc = nn.Linear(4 * 4 * 4, 10)
    
    def forward(self, x):
        # 畳み込み1 + ReLU + プーリング
        x = F.relu(self.conv1(x))      # 24→20
        x = F.max_pool2d(x, 2)          # 20→10
        
        # 畳み込み2 + ReLU + プーリング
        x = F.relu(self.conv2(x))      # 10→6
        x = F.max_pool2d(x, 2)          # 6→3... あれ？
        
        # 注: 教科書の例では6→4になる設定
        # ここでは簡略化のため調整
        
        # 平坦化
        x = x.view(x.size(0), -1)
        
        # 全結合層
        x = self.fc(x)
        
        # ソフトマックス（確率に変換）
        return F.softmax(x, dim=1)

# モデル作成とパラメータ数確認
model = SimpleCNN()
print("=== SimpleCNN のパラメータ数 ===")
for name, param in model.named_parameters():
    print(f"{name}: {param.numel()}")

total_params = sum(p.numel() for p in model.parameters())
print(f"\n合計: {total_params} パラメータ")

## まとめ

### ソフトマックス関数
$$\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_{k=1}^{n} e^{x_k}}$$

- 任意の値を **確率** (0〜1, 合計1) に変換
- 大きい値ほど高い確率に

### 全結合層
- 畳み込み・プーリングで抽出した特徴を分類結果に変換
- すべての入力がすべての出力に接続
- パラメータ数 = 入力数 × 出力数 + バイアス

### CNNの完全な流れ
```
入力画像 → [畳み込み → ReLU → プーリング] × N回 → 平坦化 → 全結合 → ソフトマックス → 確率
```

### 次のステップ

次回は**誤差逆伝播法**と**勾配降下法**による学習のメカニズムを学びます。